In [1]:
import openai
from openai import OpenAI
import os

In [2]:
API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=API_KEY)

In [3]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{"role":"user","content":"is it too late to join the course?"}]   
)

In [4]:
response.choices[0].message.content

"It depends on the specific course you're referring to. Many courses have specific enrollment deadlines, but some may allow late registration or have rolling admissions. You should check the course's official website or contact the course organizers directly for the most accurate information regarding enrollment options."

## Question 1:

Run Elastic Search 8.4.3, and get the cluster information. What's the version.build_hash value?

## Solution:

If you run it on localhost, this is how you do it:

docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3

curl localhost:9200

Soln:

"build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73"

## Question 2:

Index the data as shown in the course. 
Make the course field a keyword with the rest as text.

Which function do you use for adding your data to elasticsearch?

insert | index | put | add

In [5]:
# To get the FAQ data,run this snippet:

import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [6]:
from elasticsearch import Elasticsearch

#Create an elasticsearch client

es_client = Elasticsearch('http://localhost:9200')

# Check all is working well. i.e That we can actually connect to elasticsearch on Docker from our Jupyter notebook! 
es_client.info()

ObjectApiResponse({'name': '29ffe0d612fb', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'X0aL8FV8Tlu79zrk_uv-qA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [10]:
query = 'I just discovered the course. Can I still join it?'

In [11]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs


In [12]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

## Q3. Searching

Now let's search in our index.

We will execute a query "How do I execute a command in a running docker container?".

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

What's the score for the top ranking result?

94.05  
84.05  
74.05  
64.05 
  
**Hint**: Look at the "_score field".

In [36]:
query = 'How do I execute a command in a running docker container?'

def elastic_search_q3(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    return response

elastic_search_q3(query)['hits']['hits'][0]['_score']

84.050095

## Q4. Filtering

Now let's only limit the questions to machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?

How do I debug a docker container?  
How do I copy files from a different folder into docker container’s working directory?  
How do Lambda container images work?  
How can I annotate a graph?

In [38]:
query = "How do I execute a command in a running docker container?"

def elastic_search_q4(query):

    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs


elastic_search_q4(query)

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)

context_template = """  
Q: {question}  
A: {text}  
""".strip() 
  
Now use the context you just created along with the "How do I execute a command in a running docker container?" question to construct a prompt using the template below:

prompt_template = """  
  
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}  

CONTEXT:  
{context}  
""".strip()  
  
What's the length of the resulting prompt? (use the len function)  

962  
1462  
1962  
2462  

In [39]:
# Records returned from Elasticsearch in Q4

records = elastic_search_q4(query)

# Context template

context_template = """
Q: {question}
A: {text}
""".strip()

# Function to build context

def build_context(records):
    context_entries = []
    for record in records:
        context_entry = context_template.format(question=record["question"], text=record["text"])
        context_entries.append(context_entry)
    return "\n\n".join(context_entries)

# Build the context from the records
context = build_context(records)

# Question to be answered

question = "How do I execute a command in a running docker container?"


# Prompt template

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()


# Construct the final prompt
prompt = prompt_template.format(question=question, context=context)

# Print the constructed prompt
print(prompt)



You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [40]:
len(prompt)

1462

## Q6

Let's calculate the number of tokens in our query.

Using the encode function, find how many tokens our prompt have has.

In [45]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [46]:
# Encode the prompt
input_ids = encoding.encode(prompt)

# Count the tokens
token_count = len(input_ids)

print(f"The prompt has {token_count} tokens.")

The prompt has 322 tokens.


## Bonus: 

Let's send the prompt to OpenAI. What's the response?

In [47]:
# Send the prompt to OpenAI
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}]
)

# Get the response
response = completion.choices[0].message.content

print(response)

To execute a command in a running Docker container, you can use the `docker exec` command. Here are the steps:

1. First, find the container ID of the running container using the following command:
   ```
   docker ps
   ```
   This command will list all running containers and their IDs.

2. Once you have the container ID, you can execute a command in the running container by using:
   ```
   docker exec -it <container-id> bash
   ```
   Replace `<container-id>` with the actual ID of your running container.

This will start an interactive bash session inside the running container, allowing you to execute commands within it.
